In [1]:
#Imports
#Imports
import numpy as np
import pandas as pd
import seaborn as sns
import ast, json
import math
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
from statsmodels.stats import weightstats as stests
import scipy.stats as scs
import statsmodels.api as sm
import statsmodels.stats as stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
from sklearn import linear_model
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
#Useful methods
def get_elems_list(row, cols):
    elem_comb = {}
    elem_list = []
    i = 0
    for a in row:
        if not math.isnan(a):
            elem_list.append(cols[i])
        i+=1
    elem_comb['elements'] = elem_list
    return elem_comb

def get_data_rep(df):
    elements = get_elements(df)
    ecols = elements.columns
    data = df.drop(columns = ecols)
    e = pd.DataFrame()
    for i in range (0, len(elements)):
        e =e.append(get_elems_list(elements.loc[i], ecols), ignore_index=True)
    print(e)
    data['elements'] = e['elements']
    return data

def get_oth_props(df):
    elements = get_elements(df)
    ecols = elements.columns
    data = df.drop(columns = ecols)
    return data
def get_chi_corrs(df, func, ind):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            try:
                a  = []
                a.append(r)
                a.append(c)
                pvalues[r][c] = round(func(df[a])[ind], 4)
            except:
                print('err here')
    return pvalues

def run_on_df(df, f):
    a = pd.DataFrame()
    for c in df.columns:
        a[c] = f(df[c],0)
    return a

def get_heatmap(df):
    return df.style.background_gradient(cmap='magma') 

def get_corrs(df, func, ind):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(func(df[r], df[c])[ind], 4)

    return pvalues

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

def notnulls(data, col):
    return data[data[col].notnull()][col]

def get_counts_df(data):
    return pd.DataFrame(data.count()).transpose()

def numerify_df(data):
    for c in data.columns:
        try:
            data[c] = data[c].astype(float)
        except:
            print(c)
    return data

def drop_txt_rows(data):
    for col in data.columns:
        if data[col].dtype ==object:
            data = data.drop(col, axis=1)  
    data = data.fillna(0)
    return data

def get_elements(df):
    new_cols = []
    for c in df.columns:
        if len(c)<3:
            new_cols.append(c)
            
    return df[new_cols]

def get_pwr_corrs(df, func):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            if r==c:
                pvalues[r][c]=0
            else:
                pvalues[r][c] = round(np.mean(get_power(df, r, c, func)[0]), 4)

    return pvalues
            

In [3]:
df = pd.read_csv("final_numeric_data.csv").drop(columns = {'Unnamed: 0'})
columns = list(df.columns)
elems = get_elements(df)
elems = numerify_df(elems).astype(float).to_numpy()
elems[elems==0] = np.nan
df = numerify_df(df).astype(float).to_numpy()
df[df==0] = np.nan

/home/prasann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
def get_model_err(df,prop):
    tmp = np.concatenate([elems, df[:,columns.index(prop)].reshape(3255,1)], axis=1)
    tmp[tmp==0] = np.nan
    tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    X = np.nan_to_num(tmp)
    np.delete(X, 2, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
    lin_reg = linear_model.LinearRegression()
    print(y_train)
    lin_reg.fit(X_train, y_train)
    pred = lin_reg.predict(X_test)
    print(pred)
    err = mean_squared_error(y_test, pred)
    return err

In [18]:
get_model_err(df, "Category")

[ 4.  2.  4.  4.  6.  9.  4.  4.  3.  3.  2.  2.  3.  9.  2.  4.  2.  4.
  9.  3.  4.  2.  2.  3. 12.  2. 12.  2.  2.  4.  9.  5.  4.  5.  2.  3.
  3.  4. 12.  4.  2. 12.  4.  4.  9.  9.  2.  3.  9.  2.  3.  9.  3.  9.
  3.  2.  2.  4.  1.  2.  4. 10.  2.  2.  2.  3.  2.  2.  9.  4.  4.  9.
  4.  2.  9.  3. 10.  2.  2.  2. 12.  2.  3.  2.  3.  8.  2.  2. 11.  4.
  2.  4.  2.  2.  5.  3.  3.  4. 10.  2.  3. 12.  2.  5.  2.  3.  4.  4.
  4.  2.  5. 12.  2.  3.  3.  2.  5.  5.  2.  2.  2.  2.  4.  3.  4.  2.
  3.  3.  2.  4.  3.  4.  7.  3.  2.  2.  2.  2.  2.  9.  3.  4.  3.  4.
  2.  3.  3.  3.  4.  4.  3. 12. 10.  3.  3.  2.  4.  5.  2.  4. 12.  4.
  2. 10.  4.  2.  3.  3.  4.  7.  4.  3.  2.  5.  2.  9.  5.  3.  3.  2.
  3.  3.  4.  3.  2.  9.  3.  4.  3.  1.  3.  2.  4.  2.  1.  3.  9. 12.
  7.  3.  2.  5. 12.  4.  2.  4.  2.  3.  3.  1.  2.  4.  4.  9.  9.  3.
  3.  7.  9. 10.  3.  2.  2.  2. 12.  2.  2.  2.  3.  9.  2.  2.  2.  2.
  2.  2.  9.  3.  4.  9.  2.  2.  2.  3.  9.  2.  2

5.4788891851813595e-28

In [76]:
print(df[:,columns.index(columns[2])].reshape(3255,1))

[[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]


In [6]:
from sklearn.metrics import mean_squared_error

In [92]:
a = []
for i in range (0,16):
    print(columns[i])
    a.append(get_model_err(df, columns[i]))
a

Category
Charpy
DIN_W
Elongation
HT_Temp
Hardness_Scale
IE-20
Manual metal arc_Heat input
Manual metal arc_Interpass temperature
Redry_Time
Reduction of area
Rp0
Tensile
Ultimate tensile strength
Yield
Yield strength


[5.4788891851813595e-28,
 6.4719809424132275e-28,
 1.4905385936132228e-28,
 1.7508855442592047e-26,
 1.6320112179393337e-26,
 4.2807443109805677e-29,
 2.0209463093553492e-26,
 2.1159139457279627e-28,
 3.5211038522453164e-28,
 3.3863285757448017e-28,
 2.2584087751949823e-27,
 2.0628787073321383e-23,
 1.404858377297941e-28,
 8.440889491323971e-26,
 0.0,
 1.3670352671399195e-27]

In [7]:
elems

,Ag,Al,B,Be,C,Co,Cr,Cu,Fe,Mg,...,P,Rm,S,Si,Sn,Ti,V,W,Zn,Zr
0,NaN,NaN,NaN,NaN,0.150,NaN,0.200,NaN,NaN,NaN,...,0.035,NaN,NaN,0.75,NaN,NaN,0.08,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,0.055,NaN,0.045,NaN,NaN,NaN,...,0.015,NaN,NaN,0.51,NaN,NaN,0.02,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.120,NaN,NaN,NaN,NaN,NaN,...,0.030,NaN,0.030,0.40,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,0.074,NaN,NaN,0.02,98.017,NaN,...,0.011,NaN,0.008,0.36,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,NaN,NaN,NaN,NaN,0.048,NaN,NaN,NaN,94.732,NaN,...,0.013,NaN,0.007,0.33,NaN,NaN,NaN,NaN,NaN,NaN
3251,NaN,NaN,NaN,NaN,0.048,NaN,NaN,NaN,94.732,NaN,...,0.013,NaN,0.007,0.33,NaN,NaN,NaN,NaN,NaN,NaN
3252,NaN,NaN,NaN,NaN,0.051,NaN,NaN,NaN,94.359,NaN,...,0.013,NaN,0.007,0.36,NaN,NaN,NaN,NaN,NaN,NaN
3253,NaN,NaN,NaN,NaN,0.051,NaN,NaN,NaN,94.359,NaN,...,0.013,NaN,0.007,0.36,NaN,NaN,NaN,NaN,NaN,NaN
